# Pytorch

Pytorch es una biblioteca creada por Facebook para hacer "programación diferencial".

Una biblioteca de programación diferencial es un sistema en donde haces operaciones y el sistema se "acuerda" de todas las operaciones que hiciste, para poder sacar derivadas.

Por ejemplo, digamos que tienes `a = 5` y `b = 3` y yo le pregunto a python `a*b`. Sin problema me contestará `15`, pero una vez que hace la operación, se olvida de donde venía ese `15`. En programación diferencial no: se acuerda  de que tenía dos variables y que las multipliqué para obtener el 15, así que si luego le pregunto a pytorch por la derivada de `a*b` con respecto a `a` y con respecto a `b`, lo puede sacar (evaluado en `(5,3)`, claro!).

Además, pytorch hace álgebra lineal, claro, y programación diferencial con álgebra lineal.

La unidad básica en pytorch son los *tensores*, que son la generalización de número, vector, matriz, etc. Si hay físicos en la audiencia, me disculpo de antemano (de parte de toda la comunidad de deep learning) por llamarle tensores a esas cosas :)

Algunas notas:

- pytorch está pensado para hacer redes neuronales, pero no es necesario querer hacer redes neuronales para usarlo!
- pytorch es extremadamente rápido. En realidad cuando le dices que multiplique matrices o lo que sea, llama a código hiper-eficiente de C++.
- Puedes usar pytorch directamente en C++, pero no ganas mucho si tus cuellos de botella son las operaciones en sí que estás haciendo.
- Puedes usar pytorch en la GPU, lo cual hace mucho más rápidas las operaciones. Incluso puedes usar muchas GPUs, o un cluster de computadoras cada una con sus GPUs, etc.
- Trae muchas funciones de ayuda para leer imágenes/audio/texto, etc. etc. Es una biblioteca muy completa. Nosotros nos enfocaremos exclusivamente a la manipulación de tensores.

In [ ]:
import torch

In [ ]:
torch.cuda.is_available()

In [ ]:
a = torch.tensor([5.])

In [ ]:
a

In [ ]:
b = torch.tensor([3.])

In [ ]:
a*b

Ahora, a esos tensores no les dije que necesitaban gradientes, así que no sabe sacar derivadas ni nada. Para poder sacar derivadas, necesito decirlo a la hora de crearlos.

In [ ]:
a = torch.tensor([5.],requires_grad=True)

In [ ]:
a

In [ ]:
b = torch.tensor([3.],requires_grad=True)

In [ ]:
a*b

In [ ]:
m1 = a*b
m2 = a+b
m1.backward() # esto significa "saca derivadas de a*b con respecto a todo"
m2.backward()

In [ ]:
a

In [ ]:
a.grad # eso fue la derivada parcial de (a*b) con respecto a, que efectivamente es 3!

In [ ]:
b.grad

Ahora, torch puede trabajar con tensores de todo tipo (enteros, boleanos, etc. etc.), pero claro, solo puede sacar derivadas de los floats.

In [ ]:
torch.tensor([1,2,3,4])

In [ ]:
A = torch.tensor([[1.,2,3,4],[5,6,7,8]])

In [ ]:
A

## Propiedades

In [ ]:
A.shape

In [ ]:
A.dtype

In [ ]:
A.device

In [ ]:
A=A.to('cuda')

In [ ]:
A.device

## Operaciones

In [ ]:
A

In [ ]:
10*A

In [ ]:
A-A

In [ ]:
A/5

In [ ]:
A//5

In [ ]:
A**2

In [ ]:
torch.sin(A)

In [ ]:
A.T

In [ ]:
A*A

In [ ]:
A@A.T

In [ ]:
A@A.T

In [ ]:
torch.arange(10)

In [ ]:
torch.arange(10,20,2)

## Generar aleatorios

In [ ]:
A = torch.rand(3,5,5); A

In [ ]:
A=A.to('cuda')

In [ ]:
A

In [ ]:
B = torch.rand(3,5,5); B

In [ ]:
A+B

In [ ]:
B=B.to(A.device)

In [ ]:
A.device

In [ ]:
B.device

In [ ]:
A+B

In [ ]:
B=torch.randn(4,3,2) # normal con media 0 y desviación estándar 1

In [ ]:
B

## Indizar

In [ ]:
A = torch.rand(4,4)

In [ ]:
A.shape

In [ ]:
A

In [ ]:
A[1]

In [ ]:
A[0,0]

In [ ]:
A[:,0].shape

In [ ]:
A[1,:] # equivalente a A[1], claro

In [ ]:
A[1]

In [ ]:
A

In [ ]:
A[1:3,:2]

In [ ]:
A

In [ ]:
A[:,None,:].shape

In [ ]:
A = A[:,None,0,None,None,None]

In [ ]:
A = torch.rand(3,4,5)

In [ ]:
A.shape

In [ ]:
A[:,[2],:].shape

In [ ]:
A=A.squeeze()

In [ ]:
A[:,:,None].shape

In [ ]:
A.unsqueeze(2).shape

In [ ]:
A.shape

In [ ]:
torch.rand(1,10)

In [ ]:
torch.rand(10,1).squeeze()

## Broadcasting

Una cosa súper padre y súper confusa a veces es el concepto de broadcasting. Si una operación entre tensores no se puede hacer porque las dimensiones no corresponden, a veces se puede hacer: aumentando dimensiones de tamaño "1" (al inicio) y "repitiendo". Vamos a ver un ejemplo.

In [ ]:
A = torch.rand(2,4); A

In [ ]:
A + 3

In [ ]:
A + torch.tensor([3])

In [ ]:
torch.tensor([3]).shape

In [ ]:
A.shape

In [ ]:
A + torch.tensor([10,20,30,40])

In [ ]:
torch.tensor([10,20,30,40]).shape

In [ ]:
torch.tensor([10,20])[:,None] + A

In [ ]:
A.shape

### ¿Cómo funciona?

Las reglas son así (digamos que las dimensiones son (4,2,1,6) y (2,5,6).)

1. Se alinean las dimensiones de los vectores A LA DERECHA 

```txt
4 2 1 6
  2 5 6
```

2. Se completa con 1's aquellos que les falte algo.
```txt
4 2 1 6
1 2 5 6
```

3. Para cada dimensión, si una de ellas es un uno, entonces se convierte a la otra dimensión, repitiendo.
```txt
4 2 5 6
4 2 5 6
```

4. Si alguna dimensión no coincide, hay error. 

Ejercicio: Súmale 10 al primer renglón y 20 al segundo renglón. Escribe poquito

```txt
2 4
2 1
```

Ejercicio 2: Crea una matriz de $n\times m$ donde en la entrada (i,j) esté el número i+j. No se vale usar la palabra "for". Con arange e indizar

In [ ]:
n,m = 5, 5
X = torch.arange(n)
Y = torch.arange(m)
X = X[None,:]
Y = Y[:,None]

In [ ]:
X.shape

In [ ]:
Y.shape

In [ ]:
torch.sum(torch.abs(X-Y))

In [ ]:
X[None,:]+Y[:,None]

In [ ]:
A = torch.rand((2,4))

In [ ]:
A

In [ ]:
A + torch.tensor([10,20])[:,None]

Ejercicio: Crea un tensor de 10x5x8 (e.g. 10 matrices de 5x8) y luego multiplica la primera matriz por 0, luego la segunda por 1, etc.

In [ ]:
A = torch.randint(0,10,(10,5,8))

In [ ]:
B = A*torch.arange(10)[:,None,None]

In [ ]:
B

In [ ]:
B.to(dtype=torch.int)

### Operaciones sobre un tensor

In [ ]:
A = torch.rand(5,3); A

In [ ]:
torch.sum(A)

In [ ]:
torch.sum(A,dim=0)

In [ ]:
torch.sum(A,dim=-1)

## Máscaras

In [ ]:
A = torch.rand(3,5)

In [ ]:
A

In [ ]:
(A < 0.5)

In [ ]:
A[A<0.5]

Ejercicio: Toma todos aquellos < a 0.5 y cámbialos por 0. (háganlo de 2 maneras)

In [ ]:
A[A<0.5] = 0

In [ ]:
A

In [ ]:
B = (A>=0.5).float()

In [ ]:
B

In [ ]:
A*B

In [ ]:
A = torch.randint(0,10,size=(5,5))

In [ ]:
A

In [ ]:
A[A%2 == 0]

Fíjate que lo convierte a un tensor de dimensión 1, pero se 'acuerda' de donde viene:

In [ ]:
A[A%2 == 0] = 99.

In [ ]:
A

## Indizar avanzado

In [ ]:
A = torch.tensor([[1,2,3,4],[5,6,7,8.]])

In [ ]:
A

In [ ]:
A[:,:-1]

In [ ]:
A[:,torch.tensor([1,3])]

In [ ]:
A = torch.rand(3,3)

In [ ]:
A

In [ ]:
torch.arange(3)

In [ ]:
A[torch.tensor([0,2,1]), torch.arange(3)]

In [ ]:
A = torch.arange(36)

In [ ]:
A

In [ ]:
A = A.reshape(2,18)

In [ ]:
A.shape

In [ ]:
A

`cat`, `stack`, `repeat`

In [ ]:
A = torch.randint(0,5,(3,5)); A

In [ ]:
B = torch.randint(0,5,(7,5)); B

In [ ]:
torch.cat((A,B),dim=1)

In [ ]:
torch.cat((A,B),dim=0)

In [ ]:
torch.stack((A,A,A))

In [ ]:
torch.stack((A,A,A,A))

In [ ]:
torch.cat((A[None],A[None],A[None],A[None]),dim=0)

In [ ]:
A = torch.randint(0,4,(2,2)); A

In [ ]:
A.repeat(3,2)

`view`, `reshape`

In [ ]:
A.shape

In [ ]:
A

In [ ]:
A.view(4,2)

In [ ]:
A.reshape(4,2)

Parecen lo mismo, pero internamente no los acomoda igual en memoria. view sólo cambia las dimensiones, reshape mueve la memoria apropiadamenet para que quede contigua la cosa

In [ ]:
A.view(1,8)

In [ ]:
A.reshape(1,8)

In [ ]:
A = A.cuda()

In [ ]:
torch.zeros_like(A)

In [ ]:
torch.ones_like(A)

Lo mejor de las variantes "_like" es que los pone en el mismo dispositivo que el original (gpu, cpu)

In [ ]:
torch.rand_like(A)

In [ ]:
torch.rand_like(A.float())

In [ ]:
A = A.float()

In [ ]:
A

In [ ]:
B = A/2

In [ ]:
B[B<=2] = 0

In [ ]:
A

In [ ]:
(A<=2).float()*A

## Ejercicios

1. Dado $n$, crea un tensor de $n\times n$ con los números del 1 al $n^2$ en orden. Ejemplo:
```
[1,2,3]
[4,5,6]
[7,8,9]
```

In [ ]:
A = torch.arange(100).reshape(10,10) + 1

2. Convierte el tensor anterior a float32 y sácale raíz cuadrada.

In [ ]:
B=torch.sqrt(A.to(dtype=torch.float))

3. Divide el primer renglón entre 1, el segundo entre 2, el tercero entre 3, etc.

In [ ]:
A

In [ ]:
A.float()/torch.arange(1,11)[:,None]

4. Crea la matriz identidad de dos maneras distintas: con `eye` y sin `eye`.

In [ ]:
torch.eye(5)

In [ ]:
A = torch.arange(10) 

In [ ]:
B=A[:,None]-A[None,:] 

In [ ]:
B

In [ ]:
B[B!=0]=-1 

In [ ]:
B

In [ ]:
B+1

In [ ]:
n = 8
I = torch.arange(n)
A=torch.zeros((n,n))

In [ ]:
A[I,I] = 1

In [ ]:
A

In [ ]:
A =torch.rand((10,10))

In [ ]:
torch.solve(torch.eye(10),A )

In [ ]:
A@torch.inverse(A)

In [ ]:
A.to(dtype=torch.float)

5. Dado una matriz $A$ de $n\times m$ y un vector de enteros $b$ donde todos los números de $b$ van del 0 a m, selecciona los números en las columnas correspondientes a b y ponlos como -1. Por ejemplo, si 
$$A = \begin{bmatrix}
        0 & 1 & 2 & 3 \\
        4 & 5 & 6 & 7 \\
      \end{bmatrix}$$
y $b = [2, 0]$, entonces debe resultar en:
$$A = \begin{bmatrix}
        0 & 1 & -1 & 3 \\
        -1 & 5 & 6 & 7 \\
      \end{bmatrix}$$

In [ ]:
b = torch.randint(0,)

In [ ]:
A[torch.arange(len(A)), b]

## Visualizar

Las imágenes en pytorch se ven como tensores de dimensiones $3\times h \times w$ donde $h$ es la altura y $w$ es la anchura.

In [ ]:
import torch
import torchvision as tv
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
I = Image.open("COVID19.jpg")

In [ ]:
I

Para convertirlas a tensores, se usa una transformación que viene en `torchvision` llamada `ToTensor()`.

In [ ]:
tv.transforms.

In [ ]:
to_tensor = tv.transforms.ToTensor()

In [ ]:
X=to_tensor(I); X

In [ ]:
X.shape

La podemos regresar a una imagen de PIL con `ToPILImage`:

In [ ]:
to_image = tv.transforms.ToPILImage()
to_image(X)

In [ ]:
to_image(torch.randn(3,256,256))

### Ejercicios

1. Muestra la imagen cortada a la mitad horizontalmente, dejando la parte de arriba.
2. Muestra la imagen cortada a la mitad verticalmente, dejando la parte derecha.
3. Llena la mitad de la imagen diagonalmente (tú escoge la diagonal) con negro y muestra la imagen.

BREAK 1

4. Convierte la imagen a escala de grises (directo con pytorch, obviamente)
5. Encuentra el **color promedio** de la imagen.
6. Para cada pixel, pon como 0 aquellos valores que estén debajo del promedio (de su propio color).
7. Todos los pixeles que sean mayormente azules conviértelos a blanco y negro. Los pixeles mayormente rojos o verdes déjalos como están.

BREAK 2

8. Crea una imagen que parezca un tablero de ajedrez.
9. Crea una imagen que parezca ruido de televisión (e.g. aleatorio pero en escala de grises)
10. Crea una imagen que parezca una gradiente horizontal y luego una gradiente vertical. Por ejemplo:

## Optimización numérica

Si tienes derivadas parciales, puedes encontrar mínimos (o máximos) locales fácilmente. Pytorch trae optimizadores que hacen esto por ti.

Por ejemplo, digamos que queremos encontrar un mínimo local de $x^2-10x + 1$ o lo que sea (agregale senos, cosenos, y otras cosas complicadas si quieres hacer algo que no podrías hacer analíticamente). 

Vamos a usar **descenso por la gradiente**.

In [ ]:
def f(x): 
    return x**2 - 10*x + 1

In [ ]:
x = torch.randn(1)*10

x = x.requires_grad_(True)

In [ ]:
f(x)

In [ ]:
from torch import optim
from tqdm import tqdm

In [ ]:
optimizer = optim.SGD([x], lr=0.01, momentum=0.9)

In [ ]:
num_steps = 1000
for step in tqdm(range(num_steps)):
    optimizer.zero_grad()
    loss = f(x)
    loss.backward()
    optimizer.step()

In [ ]:
x

**Nota**: Pytorch trae una cantidad **masiva** de cosas. pueden verlo en la documentación. Cualquier cosa que quieras hacer, seguro puedes!

- [Tutoriales](https://pytorch.org/tutorials/)
- [Documentación](https://pytorch.org/docs/stable/index.html)